### 라이브러리

In [22]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [23]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [24]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [25]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [26]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [27]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [28]:
month1 = {
    1:3,
    2:1,
    3:2,
    4:6,
    5:7,
    6:8,
    7:10,
    8:9,
    9:5,
    10:4,
    11:11,
    12:12
}

month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [29]:
train['공휴일전후1'] = 0
test['공휴일전후1'] = 0

train['공휴일전후2'] = 0
test['공휴일전후2'] = 0

In [30]:
train['공휴일전후1'][4] = 1 #2
train['공휴일전후1'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후1'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후1'][152] = 1
train['공휴일전후1'][226] = 1
train['공휴일전후1'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후1'][224] = 1
train['공휴일전후1'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후1'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후1'][310] = 2
train['공휴일전후1'][311] = 1
train['공휴일전후1'][309] = 1
train['공휴일전후1'][330] = 1
train['공휴일전후1'][379] = 1
train['공휴일전후1'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후1'][470] = 1
train['공휴일전후1'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후1'][511] = 1
train['공휴일전후1'][565] = 1
train['공휴일전후1'][623] = 1
train['공휴일전후1'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후1'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후1'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후1'][815] = 2
train['공휴일전후1'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후1'][950] = 1
train['공휴일전후1'][951] = 1
train['공휴일전후1'][953] = 2
train['공휴일전후1'][954] = 1
train['공휴일전후1'][955] = 1
train['공휴일전후1'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후1'][1038] = 1
train['공휴일전후1'][1099] = 1
train['공휴일전후1'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후1'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후1'][10] =2
test['공휴일전후1'][20] = 1

In [31]:
train['공휴일전후2'][4] = 1 #2
train['공휴일전후2'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후2'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후2'][152] = 1
train['공휴일전후2'][226] = 1
train['공휴일전후2'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후2'][224] = 1
train['공휴일전후2'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후2'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후2'][310] = 2
train['공휴일전후2'][311] = 1
train['공휴일전후2'][309] = 1
train['공휴일전후2'][330] = 1
train['공휴일전후2'][379] = 1
train['공휴일전후2'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후2'][470] = 1
train['공휴일전후2'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후2'][511] = 1
train['공휴일전후2'][565] = 1
train['공휴일전후2'][623] = 1
train['공휴일전후2'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후2'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후2'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후2'][815] = 2
train['공휴일전후2'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후2'][950] = 1
train['공휴일전후2'][951] = 1
train['공휴일전후2'][953] = 2
train['공휴일전후2'][954] = 1
train['공휴일전후2'][955] = 1
train['공휴일전후2'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후2'][1038] = 1
train['공휴일전후2'][1099] = 1
train['공휴일전후2'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후2'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후2'][10] =2
test['공휴일전후2'][20] = 1





# test['공휴일전후2'][18] =2 # 석식에만 추가
# test['공휴일전후2'][42] =2 # 석식에만 추가

In [32]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후1'])
test = pd.get_dummies(test, columns=['공휴일전후1'])

train = pd.get_dummies(train, columns=['공휴일전후2'])
test = pd.get_dummies(test, columns=['공휴일전후2'])

In [33]:
test['공휴일전후1_0'][20] = 1
test['공휴일전후1_1'][20] = 0

test['공휴일전후2_0'][20] = 1
test['공휴일전후2_1'][20] = 0

In [34]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균


478.8605851979346

In [35]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 현재 1월만 대체
train['석식계'][244] = 479.8605851979346

#train['석식계'][281] = 398
train['석식계'][492] = 479.8605851979346

#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 479.8605851979346

#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 479.8605851979346

#train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

In [36]:
# train.loc[(train.요일==4), '석식계'] -= 9

### TOP Score

In [37]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [38]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월', '일','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월', '일','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','본사출장자수']]

In [39]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 204.8292996	total: 908us	remaining: 907ms
1:	learn: 199.6189667	total: 2.89ms	remaining: 1.44s
2:	learn: 195.3471560	total: 6.03ms	remaining: 2s
3:	learn: 190.6789052	total: 7.95ms	remaining: 1.98s
4:	learn: 186.7269023	total: 9.93ms	remaining: 1.98s
5:	learn: 182.9394124	total: 10.9ms	remaining: 1.81s
6:	learn: 179.7832990	total: 11.7ms	remaining: 1.66s
7:	learn: 176.0132022	total: 13.4ms	remaining: 1.66s
8:	learn: 172.2301893	total: 17.9ms	remaining: 1.97s
9:	learn: 168.4580340	total: 19.7ms	remaining: 1.95s
10:	learn: 165.0713858	total: 21.3ms	remaining: 1.92s
11:	learn: 162.0616286	total: 23ms	remaining: 1.89s
12:	learn: 158.9978911	total: 24.2ms	remaining: 1.83s
13:	learn: 156.1848628	total: 25.7ms	remaining: 1.81s
14:	learn: 153.5045617	total: 26.9ms	remaining: 1.77s
15:	learn: 151.0606807	total: 28.5ms	remaining: 1.75s
16:	learn: 148.6386700	total: 30.1ms	remaining: 1.74s
17:	learn: 146.4262354	total: 31.7ms	remaining: 1.73s
18:	learn: 144

In [40]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,1017.521299,295.817364
1,2021-01-28,989.265617,496.398225
2,2021-01-29,601.296165,191.270340


In [41]:
submission

,일자,중식계,석식계
0,2021-01-27,1017.521299,295.817364
1,2021-01-28,989.265617,496.398225
2,2021-01-29,601.296165,191.270340
3,2021-02-01,1225.422103,533.714239
4,2021-02-02,1045.865705,566.583226
5,2021-02-03,1060.138116,511.629029
6,2021-02-04,1019.678294,562.391293
7,2021-02-05,711.291508,298.574674
8,2021-02-08,1309.594750,678.219368
9,2021-02-09,1068.514763,578.691886


In [220]:
# submission.중식계 = submission.중식계-5

In [199]:
submission['석식계'].mean()

516.8180634873239

In [42]:
submission.to_csv('1대체금月.csv', index=False)

In [70]:
submission.중식계.mean()

983.7756025912

In [69]:
train[train.요일==4].석식계 = train[train.요일==4].석식계 + 20

In [79]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '중식계', '석식계', '요일(석식)', '년', '월', '일', '주', '식사가능자수',
       '월(중식)', '월(석식)', '주(중식)', '주(석식)', '공휴일전후1_0', '공휴일전후1_1', '공휴일전후1_2',
       '공휴일전후2_0', '공휴일전후2_1', '공휴일전후2_2'],
      dtype='object')

In [84]:
train.요일

0       0
1       1
2       2
3       3
4       4
       ..
1200    2
1201    3
1202    4
1203    0
1204    1
Name: 요일, Length: 1205, dtype: int64